In [3]:
import pandas as pd
import numpy as np
import cx_Oracle
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_account = pd.read_csv('account.asc', sep=';')
df_card = pd.read_csv('card.asc', sep=';')
df_client = pd.read_csv('client.asc', sep=';')
df_disp = pd.read_csv('disp.asc', sep=';')
df_district = pd.read_csv('district.asc', sep=';')
df_loan= pd.read_csv('loan.asc', sep=';')
df_order= pd.read_csv('order.asc', sep=';')
df_trans = pd.read_csv('trans.asc', sep=';')

c:\Users\cchiri\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
